In [ ]:
from langchain.chains import RetrievalQA
from langchain_groq.chat_models import ChatGroq
from langchain.document_loaders import CSVLoader
from langchain.vectorstores import DocArrayInMemorySearch
from langchain.indexes import VectorstoreIndexCreator
from langchain.embeddings import HuggingFaceEmbeddings
embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
llm = ChatGroq(api_key=os.environ["GROQ_API_KEY"], model="openai/gpt-oss-120b", temperature=0.9)

from IPython.display import display, Markdown
import os
from pprint import pprint
from dotenv import load_dotenv
load_dotenv()

c:\Users\fredr\OneDrive\Desktop\llm-utilities\llm\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\fredr\AppData\Local\Temp\ipykernel_22252\3306920769.py:7: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")


True

In [9]:
file = "products_2000.csv"
loader = CSVLoader(file_path=file)

index = VectorstoreIndexCreator(
    embedding=embedding_model,
    vectorstore_cls=DocArrayInMemorySearch
).from_loaders([loader])

In [10]:
query = "Please list the names of all of the Puma T-shirts."
response = index.query(llm=llm, question=query)
display(Markdown(response))

# This will not be efficient enough.

Here are the names (titles) of all the Puma T‑shirts in the catalog:

1. **Puma T-shirts Edition 7507**  
2. **Puma T-shirts Classic 8590**  
3. **Puma T-shirts X 8410**  
4. **Puma T-shirts X 9577**

### Populating the Vector Database with the product chunks

In [11]:
loader = CSVLoader(file_path=file)
docs = loader.load()

pprint(docs[1999].__dict__)

{'id': None,
 'metadata': {'row': 1999, 'source': 'products_2000.csv'},
 'page_content': 'product_id: 2002000\n'
                 'sku: AXE-02000\n'
                 'title: Axe Haircare (Shampoo Lite 4476\n'
                 'slug: axe-haircare-shampoo-lite-4476\n'
                 'brand: Axe\n'
                 'category: Haircare (Shampoo / Conditioner / Wax / Gel)\n'
                 'subcategory: Haircare (Shampoo\n'
                 'price: 74.59\n'
                 'discounted_price: 67.49\n'
                 'currency: USD\n'
                 'description: Axe Haircare (Shampoo Lite 4476: High-quality '
                 'Haircare (Shampoo / Conditioner / Wax / Gel) product by Axe. '
                 'Lightweight.\n'
                 'image_url: '
                 'https://picsum.photos/seed/axe-haircare-shampoo-lite-4476/1200/1200\n'
                 'image_filename: axe-haircare-shampoo-lite-4476.jpg\n'
                 'available_countries: CA,DE\n'
                 'stock: 

In [14]:
query = "Please suggest a Puma T-shirt."
query_embedding = embedding_model.embed_query(query)

db = DocArrayInMemorySearch.from_documents(docs, embedding_model)

similar_docs = db.similarity_search(query=query)
list(similar_docs)

[Document(metadata={'source': 'products_2000.csv', 'row': 166}, page_content='product_id: 2000167\nsku: PUM-00167\ntitle: Puma T-shirts Edition 7507\nslug: puma-t-shirts-edition-7507\nbrand: Puma\ncategory: T-shirts / Casual Shirts\nsubcategory: T-shirts\nprice: 183.85\ndiscounted_price: 111.11\ncurrency: USD\ndescription: Puma T-shirts Edition 7507: High-quality T-shirts / Casual Shirts product by Puma. Stylish.\nimage_url: https://picsum.photos/seed/puma-t-shirts-edition-7507/1200/1200\nimage_filename: puma-t-shirts-edition-7507.jpg\navailable_countries: US,DE,UK,FR\nstock: 290\nis_available: True\ntags: premium,outdoor,casual\ngender: Women\ncolor: White\nsize: One Size\nmaterial: Canvas\nrating: 4.8\nnum_reviews: 3366\npopularity_score: 80\nweight_kg: 2.31\ndimensions_cm: 41x43x31\nmanufactured_date: 09-10-2025\ngenre: '),
 Document(metadata={'source': 'products_2000.csv', 'row': 169}, page_content='product_id: 2000170\nsku: PUM-00170\ntitle: Puma T-shirts Classic 8590\nslug: puma-

In [15]:
similar_docs[0]

Document(metadata={'source': 'products_2000.csv', 'row': 166}, page_content='product_id: 2000167\nsku: PUM-00167\ntitle: Puma T-shirts Edition 7507\nslug: puma-t-shirts-edition-7507\nbrand: Puma\ncategory: T-shirts / Casual Shirts\nsubcategory: T-shirts\nprice: 183.85\ndiscounted_price: 111.11\ncurrency: USD\ndescription: Puma T-shirts Edition 7507: High-quality T-shirts / Casual Shirts product by Puma. Stylish.\nimage_url: https://picsum.photos/seed/puma-t-shirts-edition-7507/1200/1200\nimage_filename: puma-t-shirts-edition-7507.jpg\navailable_countries: US,DE,UK,FR\nstock: 290\nis_available: True\ntags: premium,outdoor,casual\ngender: Women\ncolor: White\nsize: One Size\nmaterial: Canvas\nrating: 4.8\nnum_reviews: 3366\npopularity_score: 80\nweight_kg: 2.31\ndimensions_cm: 41x43x31\nmanufactured_date: 09-10-2025\ngenre: ')

### Retrieval

In [16]:
retriever = db.as_retriever()
qdocs = "".join([similar_docs[i].page_content for i in range(len(similar_docs))])
response = llm.call_as_llm(f"{qdocs} Question: Please list all your \
shirts with sun protection in a table in markdown and summarize each one.")

display(Markdown(response))

**Shirts with Sun‑Protection (based on “outdoor” tag)**  

| Product ID | SKU       | Title                         | Price (USD) | Discounted Price (USD) | Color | Size      | Material | Rating | Reviews | Stock | Availability | Summary |
|------------|-----------|------------------------------|------------|------------------------|-------|-----------|----------|--------|---------|-------|--------------|---------|
| 2000167    | PUM‑00167 | Puma T‑shirts Edition 7507   | 183.85     | 111.11                 | White | One Size  | Canvas   | 4.8    | 3 366   | 290   | In stock     | Premium women’s T‑shirt designed for outdoor activities. Made of sturdy canvas, it offers a comfortable fit, high durability and likely includes UV‑blocking features (indicated by the “outdoor” tag). Ideal for casual wear with a stylish white look. |

**Note:**  
- The only shirt in the supplied list that hints at sun‑protection is the **Puma T‑shirts Edition 7507**, thanks to its `outdoor` tag.  
- None of the other shirts explicitly mention sun‑protection in their tags or descriptions. If you need more options, let me know and I can help you filter by other criteria!

### Retrieval QA Chain

Chain Types,

- stuff
- map_reduce
- refine
- map_rerank

**"stuff" method**

In [17]:
qa_stuff = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff",
    retriever=retriever, verbose=True
)
query = "Please suggest a Puma T-shirt."
response = qa_stuff.run(query)

display(Markdown(response))

C:\Users\fredr\AppData\Local\Temp\ipykernel_22252\908938858.py:6: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = qa_stuff.run(query)




> Entering new RetrievalQA chain...

> Finished chain.


Here are a few of the Puma T‑shirts we have in stock, along with the details that might help you decide which one fits you best:

| # | Product | Who it’s for | Color | Size | Material | Original Price | Discounted Price | Rating | Stock |
|---|---------|--------------|-------|------|----------|----------------|------------------|--------|-------|
| 1 | **Puma T‑shirts Classic 8590** | Men | Red | One Size | Resin | **$176.47** | **$107.67** | 4.6 ★ (4 ,137 reviews) | 127 |
| 2 | **Puma T‑shirts Edition 7507** | Women | White | One Size | Canvas | **$183.85** | **$111.11** | 4.8 ★ (3 366 reviews) | 290 |
| 3 | **Puma T‑shirts X 8410** | Men | Grey | 8 | Plastic | **$8.91** | **$7.78** | 4.7 ★ (3 760 reviews) | 114 |
| 4 | **Puma T‑shirts Lite 8947** | Men | Grey | M | Leather | **$78.19** | **$69.60** | 3.7 ★ (3 075 reviews) | 37 |

### Quick recommendation

- **If you want a high‑rated, premium‑look tee for women** – go with **Puma T‑shirts Edition 7507** (white, canvas, 4.8★).  
- **If you prefer a popular, sporty red tee for men** – **Puma T‑shirts Classic 8590** is a solid choice (4.6★, high popularity).  
- **If you’re looking for an ultra‑budget option** – the **Puma T‑shirts X 8410** costs under $8 after discount and still has a strong 4.7★ rating.  

Let me know if you have a specific gender, color, size, or price range in mind, and I can narrow it down further!

In [18]:
response = index.query(query, llm=llm)
index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch,
    embedding = embedding_model
).from_loaders([loader])